In [298]:
# Understanding supervised learning
# Supervised - Data points have known outcomes
# Unsupervised - Data points have unknown outcomes

# Regression - Output is numerical (Continous)
# Classification - Output is categorical

# Data with answers + Model -> Fit (Gives an Equation)
# Data without answers + Fit_Model -> Predict -> Predicted Answers

# Dataframes and data structures for Data Science
import pandas as pd

# Mathematical computation /if required
import numpy as np

In [299]:
# Data prerocessing with Sklearn
# Data scalars - These are your buddies for algorithms that use distance based calculations, if so then the scaling of
# diferent kinds of attributes needs to be done. Different kinds of scaling options are avalibale.

# 1. Standard Scaling - Scales from the mean in terms of UNIT VARIANCE
# 2. Minimun - Maximum Scaling - Scale to fixed range data e.g. - (5-10)
# 3. Maximum Absolute Value Scaling - Scale from 0 to maximum number to bring it between 0s and 1s, we should be
#    careful with negative values!

# STANDARD SCALAR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

In [301]:
# Create the scalar object

stdSc = StandardScaler()
minMax = MinMaxScaler()
absSc = MaxAbsScaler()

In [335]:
# KNN Classification
# Stores all available cases -> new data point -> case based on a similarity

# k - Number of nearest neighbours

# KNN - Recommender system
#     - Document Filtering - Finding the concept

# Knn working - Similarity based algorithm
#             - Assign a point, ask which category it belongs to class b?
#             - Increasing k, the radius increases
#             - How to choose k? Trial and Error if 1000 then 1-19 is ideal k value
#             - L2 : Eucledian distance (Direct Distance)
#             - L1: Manhattan distance (Roads Distance) Adding absolute deltas

# Lazy Learner - Tries to memorize the data, only during prediction, tries to plot the point 

# Let's use the Iris Data Set as an example

# Telecom Customer Churn Prediction Data
# Import the dataset 
df = pd.read_csv('Data/Orange_Telecom_Churn_Data.csv')

# Unnecessary categorical data
df = df.drop('phone_number', 1)

# Creating dummy columns
df = dummyAndBindBinary(df,'voice_mail_plan')
df = dummyAndBindBinary(df,'intl_plan')
df = dummyAndBind(df,'state')

df = attachTargetAtTheEnd(df, 'churned')
df.head()

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,...,TN,TX,UT,VA,VT,WA,WI,WV,WY,churned
0,128,415,25,265.1,110,45.07,197.4,99,16.78,244.7,...,0,0,0,0,0,0,0,0,0,False
1,107,415,26,161.6,123,27.47,195.5,103,16.62,254.4,...,0,0,0,0,0,0,0,0,0,False
2,137,415,0,243.4,114,41.38,121.2,110,10.30,162.6,...,0,0,0,0,0,0,0,0,0,False
3,84,408,0,299.4,71,50.90,61.9,88,5.26,196.9,...,0,0,0,0,0,0,0,0,0,False
4,75,415,0,166.7,113,28.34,148.3,122,12.61,186.9,...,0,0,0,0,0,0,0,0,0,False


In [331]:
# Defining an easy function for dummyfying and binding

def dummyAndBindBinary(df, attribute):
    dfvm = pd.get_dummies(df[attribute])
    dfvm.columns = [attribute + '_no',attribute + '_yes']
    df = pd.concat([df, dfvm], axis = 1)
    df = df.drop([attribute], axis = 1)
    return df

def dummyAndBind(df, attribute):
    dfvm = pd.get_dummies(df[attribute])
    df = pd.concat([df, dfvm], axis = 1)
    df = df.drop([attribute], axis = 1)
    return df

def attachTargetAtTheEnd(df, target):
    tarCol = df[target]
    df = df.drop([target], axis = 1)
    df = pd.concat([df, tarCol], axis = 1)
    return df
    

In [392]:
# Split into test and train data
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

# Use the Knn Algorithm
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=7)
KNN = KNN.fit(X=train.iloc[:,:-1],y=train.iloc[:,-1])

In [393]:
predicted = pd.Series(KNN.predict(X=test.iloc[:,:-1]))

# Because of random selection of indexes, jumbled indexes are present
predicted.index = test.index
test['churned_knn'] = predicted

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [394]:
test.head(5)

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,...,TX,UT,VA,VT,WA,WI,WV,WY,churned,churned_knn
3389,83,510,0,129.6,81,22.03,189.7,117,16.12,219.0,...,0,0,0,0,0,0,0,0,False,False
4388,93,415,21,213.8,103,36.35,257.7,72,21.90,218.4,...,0,0,0,0,0,0,0,0,False,False
1084,111,415,0,255.9,97,43.50,204.1,129,17.35,171.3,...,0,0,0,0,0,0,0,0,False,False
4343,73,510,0,159.0,123,27.03,233.3,95,19.83,178.9,...,0,0,0,0,0,0,0,0,False,False
3734,110,415,0,198.2,92,33.69,147.1,97,12.50,199.6,...,0,0,0,0,0,0,0,0,False,False


In [395]:
# Accuracy calculation
from sklearn import metrics
print("Model Accuracy on Test:",int(metrics.accuracy_score(test['churned'], test['churned_knn'])*100),"%")

Model Accuracy on Test: 88 %
